In [7]:
from pathlib import Path

import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM

print("Imported Libraries")

Imported Libraries


In [8]:
DES_HATS_DIR = Path("../../../shared/hats/catalogs/des/des_dr2")
GAIA_HATS_DIR = Path("../../../shared/hats/catalogs/gaia_dr3/gaia")

CATALOG_DIR = Path("../../catalogs")

GAIA_PM_NAME = "gaia_dr3_pm_greater_100"
GAIA_PM_DIR = CATALOG_DIR / GAIA_PM_NAME

DES_LIGHT_NAME = "des_light"
DES_LIGHT_DIR = CATALOG_DIR / DES_LIGHT_NAME

XMATCH_NAME = "des_dr2_x_gaia_dr3"
XMATCH_DIR = CATALOG_DIR / XMATCH_NAME

print("Defined directories")

Defined directories


In [9]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
print("Defined filter variables")

Defined filter variables


In [10]:
%%time

with Client(n_workers=4):
    lsdb.read_hats(
        path=GAIA_HATS_DIR,
        columns=['ra', 'pmra', 'dec', 'pmdec', 'source_id', 'parallax_over_error'],
    ).query('pmra*pmra + pmdec*pmdec > 100*100').to_hats(catalog_name = GAIA_PM_NAME, 
                                                        base_catalog_path = GAIA_PM_DIR)

print("gaia_pm successfully saved to /catalog")

2025-05-21 16:21:31,670 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-05-21 16:21:31,690 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-05-21 16:21:31,691 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-05-21 16:21:31,691 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing


gaia_pm successfully saved to /catalog
CPU times: user 58.7 s, sys: 3.23 s, total: 1min 1s
Wall time: 2min 55s


In [ ]:
%%time

with Client(n_workers=4) as client:
    lsdb.read_hats(
        DES_HATS_DIR,
        columns=des_cols,
    ).to_hats(catalog_name = DES_LIGHT_NAME, 
                base_catalog_path = DES_LIGHT_DIR)